In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta

# Define the FRED API key and base URL
fred_api_key = 'bdb4f5e5fc848858bd5578696d6b810e'
base_url = 'https://api.stlouisfed.org/fred/series/observations'

# Define a function to fetch data from FRED API
def fetch_fred_data(series_id, start_date, end_date):
    params = {
        'series_id': series_id,
        'api_key': fred_api_key,
        'file_type': 'json',
        'observation_start': start_date,
        'observation_end': end_date
    }
    response = requests.get(base_url, params=params)
    data = response.json()
    return pd.DataFrame(data['observations'])

# Define the series IDs for the 6 GDP components
series_ids = {
    'consumer_spending': 'PCE',
    'private_inventories': 'A014RE1Q156NBEA',
    'nonresidential_investment': 'PNFI',
    'government_spending': 'GCE',
    'residential_investment': 'PRFI',
    'net_exports': 'NETEXP'
}

# Define the start and end dates
start_date = (datetime.now() - timedelta(days=25*365)).strftime('%Y-%m-%d')
end_date = datetime.now().strftime('%Y-%m-%d')

# Fetch data for each GDP component
component_data = {}
for component, series_id in series_ids.items():
    component_data[component] = fetch_fred_data(series_id, start_date, end_date)

# Display the head of each component's data
display_data = {component: data.head() for component, data in component_data.items()}
display_data

{'consumer_spending':   realtime_start realtime_end        date   value
 0     2024-09-14   2024-09-14  1999-09-01  6386.5
 1     2024-09-14   2024-09-14  1999-10-01  6411.4
 2     2024-09-14   2024-09-14  1999-11-01  6448.7
 3     2024-09-14   2024-09-14  1999-12-01  6541.0
 4     2024-09-14   2024-09-14  2000-01-01  6542.9,
 'private_inventories':   realtime_start realtime_end        date value
 0     2024-09-14   2024-09-14  1999-07-01   0.4
 1     2024-09-14   2024-09-14  1999-10-01   0.9
 2     2024-09-14   2024-09-14  2000-01-01   0.2
 3     2024-09-14   2024-09-14  2000-04-01   0.9
 4     2024-09-14   2024-09-14  2000-07-01   0.6,
 'nonresidential_investment':   realtime_start realtime_end        date     value
 0     2024-09-14   2024-09-14  1999-07-01  1386.594
 1     2024-09-14   2024-09-14  1999-10-01  1395.049
 2     2024-09-14   2024-09-14  2000-01-01   1450.25
 3     2024-09-14   2024-09-14  2000-04-01  1498.729
 4     2024-09-14   2024-09-14  2000-07-01  1519.657,
 'gove

In [2]:
# Import necessary libraries for MIDAS regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Define a function to perform MIDAS regression
# Note: This is a simplified version of MIDAS regression using linear regression for demonstration purposes

def midas_regression(X, y):
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Initialize and fit the linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Return the model and predictions
    return model, y_pred

# Prepare the data for MIDAS regression
# For simplicity, let's assume we have monthly data for the independent variable
# Here, we will use the 'value' column from each component's data as the dependent variable

# Example for consumer spending
consumer_spending_data = component_data['consumer_spending']
consumer_spending_data['value'] = pd.to_numeric(consumer_spending_data['value'], errors='coerce')
X_consumer_spending = np.arange(len(consumer_spending_data)).reshape(-1, 1)  # Dummy independent variable

y_consumer_spending = consumer_spending_data['value']

# Perform MIDAS regression for consumer spending
model_consumer_spending, y_pred_consumer_spending = midas_regression(X_consumer_spending, y_consumer_spending)

# Print the model coefficients and predictions for consumer spending
print('Consumer Spending Model Coefficients:', model_consumer_spending.coef_)
print('Consumer Spending Predictions:', y_pred_consumer_spending[:5])

# Repeat the process for other GDP components
# Note: In a real-world scenario, you would use actual monthly data for the independent variable

# Done for consumer spending, similar steps can be repeated for other components
print("MIDAS regression for consumer spending completed.")

Consumer Spending Model Coefficients: [38.39107258]
Consumer Spending Predictions: [16572.29395502 15958.03679377 12080.53846339  6129.9222138
  8740.5151491 ]
MIDAS regression for consumer spending completed.


In [3]:
# Perform MIDAS regression for other GDP components

# Function to perform MIDAS regression for each component

def perform_midas_for_component(component_name):
    data = component_data[component_name]
    data['value'] = pd.to_numeric(data['value'], errors='coerce')
    X = np.arange(len(data)).reshape(-1, 1)  # Dummy independent variable
    y = data['value']
    model, y_pred = midas_regression(X, y)
    print(component_name.capitalize() + ' Model Coefficients:', model.coef_)
    print(component_name.capitalize() + ' Predictions:', y_pred[:5])

# List of components to process
components = ['private_inventories', 'nonresidential_investment', 'government_spending', 'residential_investment', 'net_exports']

# Perform MIDAS regression for each component
for component in components:
    perform_midas_for_component(component)

print("MIDAS regression for all components completed.")

Private_inventories Model Coefficients: [0.00136862]
Private_inventories Predictions: [0.29163799 0.25057939 0.27384593 0.23963044 0.23826182]
Nonresidential_investment Model Coefficients: [23.97528979]
Nonresidential_investment Predictions: [3042.45165827 2323.19296453 2730.77289098 2131.3906462  2107.41535641]
Government_spending Model Coefficients: [27.47942458]
Government_spending Predictions: [4049.85775532 3225.47501781 3692.62523573 3005.63962115 2978.16019656]
Residential_investment Model Coefficients: [5.29374526]
Residential_investment Predictions: [872.37242215 713.56006434 803.55373376 671.21010225 665.91635699]
Net_exports Model Coefficients: [-2.86899494]
Net_exports Predictions: [-699.02912437 -612.95927612 -661.73219013 -590.00731659 -587.13832164]
MIDAS regression for all components completed.
